In [75]:
import requests as req
import json
from time import sleep
import pandas as pd
import seaborn as sns
%matplotlib inline

The following code below creates a list of unique id numbers for each NBA team in sports radar database.


In [57]:
def get_player_avgs():
    rankings=req.get('https://api.sportradar.us/nba/trial/v5/en/seasons/2013/REG/rankings.json?api_key=ps3d3xept7zttv9ac8v85ej4').json()
    sub_info=[]
    for i in rankings:
        if i=='conferences':
            for e in rankings[i]:
                for gen_info in e['divisions']:
                    for team_info in gen_info['teams']:
                        sub_info.append(team_info['id'])           
    player_avgs={}
    for year in range(2013,2019):
        year=str(year)
        player_avgs[year]={}
        for i_d in sub_info:
            player_info={}
            team_info=req.get('https://api.sportradar.us/nba/trial/v5/en/seasons/{}/REG/teams/{}/statistics.json?api_key=ps3d3xept7zttv9ac8v85ej4'.format(year,i_d))
            team_info=team_info.json()
            sleep(1.5) ##the program is paused here to void making more calls to the sports radar API than i'm allowed per second
            for record in team_info:
                if record=='players':
                    for data in team_info[record]:
                        player_info[data['full_name']]=data['average']
                    player_avgs[year][team_info['name']]=player_info
    return pd.DataFrame(player_avgs)
    
def team_generator():
    df=get_player_avgs()
    pretrain={}
    for season in df:
        lst=[]
        for team in df[season]:
            result=pd.DataFrame(team).T
            lst.append(result)
        pretrain[season]=pd.concat(lst,copy=False,)
    return pretrain            

def make_labels():
    df_mvp={'2018':'James Harden',
            '2017':'Russell Westbrook',
            '2016':'Stephen Curry',
            '2015':'Stephen Curry',
            '2014':'Kevin Durant',
            '2013':'LeBron James'}
    for season in train:
        mvp=[float(0) for i in range(len(train[season]))]
        train[season]['mvp']=mvp
        if season in df_mvp:
            train[season]['mvp'].loc[df_mvp[season]]=float(1)
    return train                



In [82]:
train=team_generator()
train=make_labels()
###Train is a dictionary of dataframes, with each dataframe containing individual stats for every player in the league for a specific year

In [84]:
train['2013'].head()

,assists,blocked_att,blocks,def_rebounds,efficiency,fast_break_att,fast_break_made,fast_break_pts,field_goals_att,field_goals_made,...,second_chance_made,second_chance_pts,steals,three_points_att,three_points_made,true_shooting_att,turnovers,two_points_att,two_points_made,mvp
Arnett Moultrie,0.2,0.33,0.33,2.00,4.58,0.0,0.0,0.0,3.17,1.33,...,0.0,0.00,0.67,0.00,0.00,3.35,0.58,3.17,1.33,0.0
Brandon Davies,0.5,0.22,0.16,1.39,3.63,0.0,0.0,0.0,2.51,1.06,...,0.0,0.08,0.47,0.20,0.04,2.97,0.69,2.31,1.02,0.0
Casper Ware,1.1,0.11,0.00,1.00,5.00,0.0,0.0,0.0,4.67,2.00,...,0.0,0.00,0.89,2.33,0.78,4.96,0.56,2.33,1.22,0.0
Daniel Orton,0.7,0.32,0.73,2.05,5.09,0.0,0.0,0.0,2.14,0.95,...,0.0,0.00,0.27,0.00,0.00,2.74,0.82,2.14,0.95,0.0
Darius Johnson-Odom,0.3,0.00,0.00,0.67,-2.33,0.0,0.0,0.0,2.33,0.00,...,0.0,0.00,0.33,1.00,0.00,2.63,0.67,1.33,0.00,0.0


In [66]:
def sort_duplicates():
    for season in train:
        uniq=[]
        duplicate_df=train[season][train[season].index.duplicated(keep=False)]
        for index in duplicate_df.index:
            records=duplicate_df.loc[index]
            records=(records.sum())/len(records)
            uniq.append(records)
        train[season].drop_duplicates(keep=False)
        train[season].append(uniq)
    return train
###To be updated to get duplicate players per season and aggregate their stats to remove all duplicates.